In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler


In [2]:
csv_names = ["하이테크센터", "로스쿨관", "1호관(본관)", "7호관(학생회관)", 
                "2호남관", "5호북관", "2호북관", "김현태인하드림센터", "5호동관",
                "5호남관", "인하드림센터", "9호관", "60주년기념관", "서호관"]

In [3]:
scaler = MinMaxScaler()

for building in (csv_names):
    df = pd.read_csv(f'OriginCSV/{building}.csv')
    
    df['DateTime'] = pd.to_datetime(df['DateTime'])
    df = df.sort_values('DateTime')
    
    df.drop('TimeStamp', axis=1, inplace=True)
    
    df['DayOfWeek'] = df['DateTime'].dt.dayofweek
    
    first_row_day_of_week = 1  # 화요일
    shift = first_row_day_of_week - df.loc[0, 'DayOfWeek']
    df['DayOfWeek'] = (df['DayOfWeek'] + shift) % 7

    df['Time'] = df['DateTime'].dt.time

    df.reset_index(drop=True, inplace=True)
    for i in range(1, len(df)-1):
        if df.loc[i, 'DataValue'] < df.loc[i-1, 'DataValue']:
            median_value = np.median([df.loc[i-1, 'DataValue'], df.loc[i+1, 'DataValue']])
            df.loc[i, 'DataValue'] = median_value

    df['PowerUsage'] = df['DataValue'].diff().fillna(0)
    df.loc[0, 'PowerUsage'] = 0  # 첫 번째 행 처리
    
    df['OriginalPowerUsage'] = df['PowerUsage']

    df['PowerUsage'] = scaler.fit_transform(df[['PowerUsage']])
    df.to_csv(f'GeneratedCSV/Preprocessed_{building}.csv', index=False)